# Configuring IP

This example notebook shows how a user can configure the IP address of their board with UDP or FTDI. 

**Refer to *Networking* in the *Advanced* section of the Naluscope manual for assistance with setting up an Ethernet connection to your board.**

## Naludaq Version

MinVersion: 0.33.2

In [ ]:
# Print Naludaq version
import naludaq

print(f"Naludaq version: {naludaq.__version__}")

## Compatible Boards

* This example applies to board models with USB or Ethernet ports.

## Imports and Variables

In [ ]:
from naluconfigs import get_available_models
from naludaq.board import Board
from naludaq.communication import ControlRegisters
from naludaq.controllers import get_connection_controller

## Logging

The logger setup below will show additional information from the NaluDAQ package. This is useful for debugging purposes, but
may be skipped if not needed.

In [ ]:
import logging


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger


logger = setup_logger()

try:
    logger.debug("logger already setup")
except Exception:
    logger = setup_logger()

In [ ]:
import time


def toggle_register(
    board: Board,
    register: str,
    delay: float = 0.1,
):
    """Toggle a control register."""
    ControlRegisters(board).write(register, 1)
    time.sleep(delay)
    ControlRegisters(board).write(register, 0)

## Options

A list of valid board models can be found using the following code snippet:


In [ ]:
# Determining Board Model
print(get_available_models())


The following parameters must be provided to successfully set the IP address:

- **board_model**:       *board_model*
  - Valid options can be found using the command in the previous code block.

- **connection_method**:      *connection_method*
  - This should be "udp" or "ftdi". Other connections methods may be possible but are not supported or implemented in this notebook.

- **default_board_addr**: (*ip_address*, *port*)
  - This is the preprogrammed board address. Refer to the Quick Start guide for your board.

- **target_board_addr**:  (*ip_address*, *port*)
  - This is the IP address that you would like to set for the board

- **recv_addr**:          (*ip_address*, *port*)
   - This is the IP address of the computer used to configure the IP address of the board

Set these parameters in the configuration block below. All file and folder names are assumed to be in the same directory as the notebook.

**Note: The code in this notebook is only valid for changing from the board's default IP address to a different IP address and does not support changing to a different IP address from a previous manually configured IP address.**

In [ ]:
################
# Board Config #
################
board_model: str = "hdsocv2_evalr2"
connection_method: str = "udp"

# Validate connection methods
connection_methods: list[str] = ["udp", "ftdi"]
if connection_method.lower() not in connection_methods:
    print(f"Invalid connection method. Valid connection methods: {connection_methods}")


#############
# IP Config #
#############
target_board_addr: tuple[str, int] = ("192.168.1.63", 4660)
recv_addr: tuple[str, int] = ("192.168.1.165", 4660)

##############
# MAC Config #
##############
target_mac_addr: int = 0x0234_1234_1234

##############
# UDP Config #
##############
default_board_addr: tuple[str, int] = ("192.168.1.58", 4660)

###############
# FTDI Config #
###############
serial_number: str = "A503VYQF"

Loads the default register and clock configuration.

In [ ]:
board = Board(board_model)

### Open connection to board. 

Connection method is specified in the Options section of this notebook.

In [ ]:
if connection_method == "udp":
    board.get_udp_connection(default_board_addr, recv_addr)
    board.disconnect()
elif connection_method == "ftdi":
    board.get_ftdi_connection(serial_number=serial_number, baud=115200)
    board.disconnect()
else:
    print(f"{connection_method} is not valid or is not implemented in this notebook.")

### Change IP address:

#### Quick guide:

(destination address, dest port), (source address, source port), mac_address

1. Write user_address registers: 
Change IP address by first changing the user address, this will update the registers but not latch the value in place.
    - `eth_dest_addr15_0`
    - `eth_dest_addr31_16`
    - `eth_src_addr15_0`
    - `eth_src_addr31_16`
    - `eth_dest_port`
    - `eth_src_port`
    - `eth_mac_addr47_32`
    - `eth_mac_addr31_16`
    - `eth_mac_addr15_0`
2. Toggle user_address_update register
Once the user address is updated, toggle the user address update registers to latch the values so the new IP address can take effect. Note that if sel is set to 1, the address takes effect immediately after the toggle.
    - `eth_src_addr_toggle`
    - `eth_dest_addr_toggle`
    - `eth_src_port_toggle`
    - `eth_dest_port_toggle`
    - `eth_mac_addr_toggle`
3. Enable select user address register
Once the new address is updated and toggled the values can be selected and the new address configurations can take effect.
    - `eth_src_addr_sel`
    - `eth_src_port_sel`
    - `eth_dest_addr_sel`
    - `eth_dest_port_sel`
    - `eth_mac_addr_sel`
4. wait
5. test

#### Change source address

In [ ]:
src_ip, src_port = target_board_addr

src_octets = get_connection_controller(board)._get_octets_or_raise(src_ip)

eth_src_addr31_16 = (src_octets[0] << 8) | src_octets[1]
eth_src_addr15_0 = (src_octets[2] << 8) | src_octets[3]
ethernet_src_config = {
    "eth_src_addr31_16": eth_src_addr31_16,
    "eth_src_addr15_0": eth_src_addr15_0,
    "eth_src_port": src_port,
}

with board:
    # 1. Change the address
    ControlRegisters(board).write_many(ethernet_src_config)

    # 2. Toggle the register to apply the change
    toggle_register(board, "eth_src_addr_toggle")
    toggle_register(board, "eth_src_port_toggle")

    # 3. Select the new address
    ControlRegisters(board).write("eth_src_addr_sel", 1)
    ControlRegisters(board).write("eth_src_port_sel", 1)

#### Change destination address

In [ ]:
dest_ip, dest_port = recv_addr

dest_octets = get_connection_controller(board)._get_octets_or_raise(dest_ip)

eth_dest_addr31_16 = (dest_octets[0] << 8) | dest_octets[1]
eth_dest_addr15_0 = (dest_octets[2] << 8) | dest_octets[3]
ethernet_dest_config = {
    "eth_dest_addr31_16": eth_dest_addr31_16,
    "eth_dest_addr15_0": eth_dest_addr15_0,
    "eth_dest_port": dest_port,
}

with board:
    # 1. Change the address
    ControlRegisters(board).write_many(ethernet_dest_config)

    # 2. Toggle the register to apply the change
    toggle_register(board, "eth_dest_addr_toggle")
    toggle_register(board, "eth_dest_port_toggle")

    # 3. Select the new address
    ControlRegisters(board).write("eth_dest_addr_sel", 1)
    ControlRegisters(board).write("eth_dest_port_sel", 1)

#### Change MAC address

In [ ]:
eth_mac_addr47_32 = (target_mac_addr >> 32) & 0xFFFF
eth_mac_addr31_16 = (target_mac_addr >> 16) & 0xFFFF
eth_mac_addr15_0 = (target_mac_addr >> 0) & 0xFFFF
ethernet_mac_config = {
    "eth_mac_addr47_32": eth_mac_addr47_32,
    "eth_mac_addr31_16": eth_mac_addr31_16,
    "eth_mac_addr15_0": eth_mac_addr15_0,
}

with board:
    # 1. Change the address
    ControlRegisters(board).write_many(ethernet_mac_config)

    # 2. Toggle the register to apply the change
    toggle_register(board, "eth_mac_addr_toggle")

    # 3. Select the new address
    ControlRegisters(board).write("eth_mac_addr_sel", 1)

# No need to a new connection, just reuse previous one

### Create a new connection with updated IP and Port

In [ ]:
new_board_addr = (src_ip, src_port)
new_recv_addr = (dest_ip, dest_port)
board.get_udp_connection(new_board_addr, new_recv_addr)

In [ ]:
with board:
    print(ControlRegisters(board).read("identifier"))

### At this point the board has a new IP, you can either create a new board object, or reuse the old one with a new connection.